# Learner for Emoji classifier 🤓
**usage:**
run all cells, then go to the [user interface](#User-Interface)

In [1]:
%matplotlib inline

In [2]:
import twitter_learning as twl
import glob
import sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import pickle
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/jonas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jonas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jonas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## user interface area:

* UI helper functions and global states

In [3]:
from IPython.display import clear_output, Markdown, Math
import ipywidgets as widgets

out_areas = {}
shown_widgets = {}
tab_manager = widgets.Tab()

def mp(obj):
    display(Markdown(obj))

def set_widget_visibility(widget_names, visible=True):
    for w in widget_names:
        shown_widgets[w].disabled = not visible

def create_area(area_name:str, list_widgets:list, out_name:str, tab=tab_manager):
    """
    creates a table of widgets with corresponding output area below
    
    @param area_name: title of the area
    @param list_widgets: list of tuples: (widget, name:str)
    @param out_name: name for the output area
    """
    if out_name is not None:
        out = widgets.Output()
        out_areas[out_name] = out
    h_box_widgets = []
    v_box_widgets = []
    for v in list_widgets:
        for h in v:
            if 'description' in h[0].__dir__() and h[1] is not None:
                h[0].description = h[1]
            if h[1] is not None:
                shown_widgets[h[1]] = h[0]
            h_box_widgets.append(h[0])
        v_box_widgets.append(widgets.HBox(h_box_widgets))
        h_box_widgets = []
    
    if out_name is not None:
        v_box_widgets += [out]
    tab.children = list(tab.children) + [widgets.VBox(v_box_widgets)]
    tab.set_title(len(tab.children) - 1, area_name)

* build UI

In [4]:
mp("----")
mp("## User Interface")
# create widgets
create_area("load dataset 💾",
           [
               [
                   (widgets.HTML("<b> Data Root Folder: </b> <br> setup the folder containing *.json train data "), None)
               ],
               [
                   (widgets.Text(value="./data_en/"), "root_path"),
                   (widgets.Button(), "set_path")
               ],
               [
                   (widgets.HTML("<b> Loading and preprocessing options: </b> <br> setup the range of files to load. Only_emoticons will filter out 'non-smiley' emojis, min_words is the minimum amount of words for one document. Also you can setup top-emoji filtering or only load samples containing a custom emoji set"), None)
               ],
               [
                   (widgets.IntRangeSlider(disabled=True, min=0, max=0), "file_range"),
                   (widgets.Checkbox(value=True,disabled=True), "only_emoticons"),
                   (widgets.Checkbox(value=False,disabled=True), "apply_lemmatization_and_stemming"),
                   (widgets.BoundedIntText(value=5,min=0, max=10), "min_words")
               ],
               [
                   #(widgets.BoundedIntText(value=-1,disabled=True,min=-1, max=10), "k_means_cluster")
                   (widgets.BoundedIntText(value=20,disabled=True,min=-1, max=100), "n_top_emojis"),
                   (widgets.Dropdown(options=["latest", "mean"], value="latest"), "label_criteria"),
                   (widgets.Text(value=""), "custom_emojis")
               ],
               [
                   (widgets.Button(disabled=True),"load_data")
               ]
           ],
           "load")

classifier_tab = widgets.Tab()

create_area("keras",
           [
               [
                   (widgets.IntSlider(min=0, max=10), "n_keras_layer")
               ],
               [
                   (widgets.HBox([]), "n_keras_neurons")
               ],
               [
                   (widgets.HBox([]), "keras_funcs")
               ]
           ],
           None,
           classifier_tab)

create_area("create/save/load classifier",
           [
               [
                   (classifier_tab, "classifier_tab")
               ],
               [
                   (widgets.HTML("<b> Create new Classifier: </b> <br> create a new keras classifier with layer options from above. Also a vectorizer will be trained on loaded sample data. If doc2vec is disabled, TFIDF is used"), None)
               ],
               [
                   (widgets.Checkbox(value=True),"use_doc2vec"),
                   (widgets.Checkbox(value=True),"d2v_use_pretrained"),
                   (widgets.IntText(value=100),"d2v_size"),
                   (widgets.IntText(value=8), "d2v_window"),
                   (widgets.IntSlider(value=5, min=0, max=32), "d2v_min_count")
               ],
               [
                   (widgets.Button(), "create_classifier")
               ],
               [
                   (widgets.HTML("<b> Save Classifier: </b>"), None)
               ],
               [
                   (widgets.Text(), "classifier_name"),
                   (widgets.Button(), "save_classifier")
               ],
               [
                   (widgets.HTML("<b> Load Classifier: </b>"), None)
               ],
               [
                   (widgets.Select(options=sorted(glob.glob("./*.pipeline"))), "clf_file_selector"),
                   (widgets.Text(), "clf_file"),
                   (widgets.Button(), "load_classifier")
               ]
           ],
           "create")

create_area("train classifier 🎓", 
            [
                [
                    (widgets.HTML("<b> Custom Batch Settings: </b> <br> (Ignored if batch_size is 0)"), None)
                ],
                [
                    (widgets.IntSlider(value=0,min=0,max=0), "batch_size"),
                    (widgets.FloatSlider(value=0.15, min=0, max=1), "val_split")
                ],
                [
                    (widgets.HTML("<b> Train: </b>"), None)
                ],
                [
                    (widgets.IntText(value=1), "n_epochs"),
                    (widgets.Button(),"train")
                ]
            ], 
            "train" )
create_area("playground 😎",
           [
               [
                   (widgets.HTML("<b> predict single sentence </b> <br> (uses min distance to given emojis in prediction_ground_set)"), None)
               ],
               [
                   (widgets.Text(),"test_input"),
                   (widgets.Text(value="😳😋😀😌😏😔😒😎😢😅😁😉🙌🙏😘😊😩😍😭😂"),"prediction_ground_set"),
                   (widgets.HTML("<h1>∅</h1>"),"prediction"),
               ],
               [
                   (widgets.Checkbox(),"show_sorted_list"),
                   (widgets.Button(),"show_plot")
               ],
               [
                   (widgets.HTML("<b> Test on loaded validation set: </b> <br> (performs prediction plot on all validation samples that are labeled with given emojis)"), None)
               ],
               [
                   (widgets.Text(value="😳😋😀😌😏😔😒😎😢😅😁😉🙌🙏😘😊😩😍😭😂"), "validation_emojis"),
                   (widgets.Button(),"show_validation_plot")
               ]
           ],
           "playground")

tab_manager

----

## User Interface

----
## global variables:

In [5]:
sdm = None
pm = None
tr = None

## pretty jupyter print

In [6]:
import collections
import traceback
from pprint import pprint as pp

def jupyter_print(obj, cell_w = 10, headers=None, p_type=True, ret_mdown=False, index_offset=0, list_horizontal=False):
    """
    pretty hacky function to convert arrays, lists and matrices into
    nice readable markdown code and render that in jupyter. if that is not possible
    it will use pretty print instead
    """
    try:
        ts = "**Type:** " + str(type(obj)).strip("<>") + "\n\n"
        if type(obj) == str:
            display(Markdown(obj))
        elif isinstance(obj, collections.Iterable):
            if isinstance(obj[0], collections.Iterable) and type(obj[0]) is not str:
                # we have a table
                
                if headers is None:
                    headers = [str(i) for i in range(len(obj[0]))]
                
                if len(headers) < len(obj[0]):
                    headers += [" " for i in range(len(obj[0]) - len(headers))]
                
                s = "|" + " " * cell_w + "|"
                
                for h in headers:
                    s += str(h) + " " * (cell_w - len(h)) + "|"
                s += "\n|" + "-" * (len(headers) + (len(headers) + 1) * cell_w) + "|\n"
                
                #s = ("|" + (" " * (cell_w))) * len(obj[0]) + "|\n" + "|" + ("-" * (cell_w + 1)) * len(obj[0])
                #s += '|\n'
                
                row = index_offset
                
                for o in obj:
                    s += "|**" + str(row) + "**" + " " * (cell_w - (len(str(row))+4))
                    row += 1
                    for i in o:
                        s += "|" + str(i) + " " * (cell_w - len(str(i)))
                    s+="|" + '\n'
                s += ts
                display(Markdown(s))
                return s if ret_mdown else None
            else:
                # we have a list
                
            
                if headers is None:
                    headers = ["index","value"]
                    
                index_title = headers[0]
                value_title = headers[1]
                
                s = "|" + index_title + " " * (cell_w - len(value_title)) + "|" + value_title + " " * (cell_w - len(value_title)) + "|" + '\n'
                s += "|" + "-" * (1 + 2 * cell_w) + '|\n'
                i = index_offset
                for o in obj:
                    s_i = str(i)
                    s_o = str(o)
                    s += "|" + s_i + " " * (cell_w - len(s_i)) + "|" + s_o + " " * (cell_w - len(s_o)) + "|" + '\n'
                    i+=1
                s += ts
                #print(s)
                display(Markdown(s))
                return s if ret_mdown else None
        else:
            jupyter_print([obj])
    except Exception as e:
        print(ts)
        pp(obj) 

jp = jupyter_print

## output progress printing:

In [7]:
class progress_indicator(object):
    
    def __init__(self, description="progress"):
        self.w = widgets.FloatProgress(value=0, min=0,max=1, description = description)
        display(self.w)
    def update(self, val):
        self.w.value = val
        

----
## load datasets

In [8]:
def set_path(b):
    with out_areas["load"]:
        clear_output()
        mp("----")
        files = sorted(glob.glob(shown_widgets["root_path"].value + "/*.json"))
        
        if len(files) == 0:
            sys.stderr.write("ERROR: no json files available in " + shown_widgets["root_path"].value + "\n")
            set_widget_visibility(["file_range",
                               "only_emoticons",
                               "n_top_emojis",
                               "apply_lemmatization_and_stemming",
                               "load_data"], False)
            return
            
        mp("**available files:**")
        jp(files, headers=["fileindex","filepath"])
        set_widget_visibility(["file_range",
                               "only_emoticons",
                               "n_top_emojis",
                               "apply_lemmatization_and_stemming",
                               "load_data"], True)
        shown_widgets["file_range"].min=0
        shown_widgets["file_range"].max=len(files) -1

def load_data(b):
    global sdm
    with out_areas["load"]:
        clear_output()
        mp("----")
        
        r = shown_widgets["file_range"].value
        r = (r[0], r[1] + 1) # range has to be exclusive according to the last element!
        
        p_r = progress_indicator("reading progress")
        
        lemm_and_stemm = shown_widgets["apply_lemmatization_and_stemming"].value
        
        if lemm_and_stemm:
            p_s = progress_indicator("stemming progress")
        
        emoji_mean = shown_widgets["label_criteria"].value == "mean"
        
        custom_emojis = list(shown_widgets["custom_emojis"].value)
        
        min_words = shown_widgets["min_words"].value
        
        sdm = twl.sample_data_manager.generate_and_read(path=shown_widgets["root_path"].value,
                                                    n_top_emojis=shown_widgets["n_top_emojis"].value,
                                                    file_range=range(r[0], r[1]),
                                                    n_kmeans_cluster=-1,
                                                    read_progress_callback=p_r.update,
                                                    stem_progress_callback=p_s.update if lemm_and_stemm else None,
                                                    apply_stemming = lemm_and_stemm,
                                                    emoji_mean=emoji_mean,
                                                    custom_target_emojis=custom_emojis if len(custom_emojis) > 0 else None,
                                                    min_words=min_words)
        shown_widgets["batch_size"].max = len(sdm.labels)
        
        
# linking functions with buttons:
shown_widgets["set_path"].on_click(set_path)
shown_widgets["load_data"].on_click(load_data)

## train

In [9]:
def train(b):
    global sdm
    global pm
    global tr
    with out_areas["train"]:
        clear_output()
        mp("----")
        if sdm is None or pm is None:
            sys.stderr.write("ERROR: sample data and/or classifier missing!\n")
            return
    
        batch_size = shown_widgets["batch_size"].value
        val_split = shown_widgets["val_split"].value
        n_epochs = shown_widgets["n_epochs"].value
        
        print("update train test split:")
        sdm.create_train_test_split(split=val_split)
        
        print("fit")
        
        p = progress_indicator()
        
        tr = twl.trainer(sdm=sdm, pm=pm)
        tr.fit(progress_callback=p.update, batch_size=batch_size if batch_size > 0 else None, n_epochs=n_epochs)
        

# linking:
shown_widgets["train"].on_click(train)
        

## create classifier

In [10]:
keras_acivations = [
    "softmax",
    "elu",
    "selu",
    "softplus",
    "softsign",
    "relu",
    "tanh",
    "sigmoid",
    "hard_sigmoid",
    "linear",
    "None"
]

def populate_keras_options(b):
    n_layers = shown_widgets["n_keras_layer"].value
    hbox_neurons = shown_widgets["n_keras_neurons"]
    hbox_funcs = shown_widgets["keras_funcs"]
    
    hbox_neurons.children = [widgets.IntText(description = str(i)) for i in range(n_layers)]
    hbox_funcs.children = [widgets.Dropdown(options=keras_acivations,description = str(i)) for i in range(n_layers)]
    
    #hbox_neurons.children[-1].disabled = True

def create_classifier(b):
    global sdm
    global pm
    global tr
    with out_areas["create"]:
        clear_output()
        mp("----")
        if sdm is None:
            sys.stderr.write("load a dataset first!\n")
            return
        
        chosen_classifier = classifier_tab.get_title(classifier_tab.selected_index)
        
        mp("**chosen classifier**: `" + chosen_classifier + "`")
        
        # creating the vectorizer
        vectorizer = None
        if shown_widgets["use_doc2vec"].value:
            if shown_widgets["d2v_use_pretrained"].value:
                vectorizer = pickle.load( open( "doc2VecModel.p", "rb" ) )
            else:
                vectorizer = twl.skd2v.Doc2VecTransformer(size=shown_widgets["d2v_size"].value,
                                                         window=shown_widgets["d2v_window"].value,
                                                         min_count=shown_widgets["d2v_min_count"].value)
        else:
            vectorizer=TfidfVectorizer(stop_words='english')
        
        # TODO: add more classifier options here:
        if chosen_classifier is 'keras':
            sdm.create_train_test_split()
            
            n_layers = shown_widgets["n_keras_layer"].value
            hbox_neurons = shown_widgets["n_keras_neurons"]
            hbox_funcs = shown_widgets["keras_funcs"]

            layers = []
            for i in range(n_layers):
                func = hbox_funcs.children[i].value
                if func == 'None':
                    func = None
                layers.append((hbox_neurons.children[i].value, func))
            
            # modify last layer:
            layers[-1] = (sdm.y.shape[1], layers[-1][1])
            
            mp("**layers:** ")
            jp(layers, headers=['#neurons', 'activation_func'])

            pm = stl.pipeline_manager.create_keras_pipeline_with_vectorizer(vectorizer, layers=layers, sdm=sdm, fit_vectorizer=not shown_widgets["d2v_use_pretrained"].value)

def save_classifier(b):
    global sdm
    global pm
    global tr
    with out_areas["create"]:
        clear_output()
        mp("----")
        if pm is None:
            sys.stderr.write("ERROR: create classifier first")
            return
        
        pm.save(shown_widgets["classifier_name"].value)


def update_file_selector(b):
    shown_widgets["clf_file_selector"].options = sorted(glob.glob("./*.pipeline"))

def clf_file_selector(b):
    shown_widgets["clf_file"].value = shown_widgets["clf_file_selector"].value
    update_file_selector(b)

def load_classifier(b):
    global sdm
    global pm
    global tr
    with out_areas["create"]:
        clear_output()
        mp("----")
        clf_file = shown_widgets["clf_file"].value
        pm = twl.pipeline_manager.load_from_pipeline_file(clf_file)
    

# link
shown_widgets["n_keras_layer"].observe(populate_keras_options)
shown_widgets["create_classifier"].on_click(create_classifier)
shown_widgets["save_classifier"].on_click(save_classifier)
shown_widgets["load_classifier"].on_click(load_classifier)
shown_widgets["clf_file_selector"].observe(clf_file_selector)





## plotting stuff for testing area

In [11]:
def sentiment_score(s):
    #(pos, neg, neu)^T
    return s[0] - s[1]

def plot_sentiment_space(predicted_sentiment_vectors, top_sentiments, top_emojis):
    # sentiment score axis
    top_X = np.array([sentiment_score(x) for x in top_sentiments])
    pred_X = np.array([sentiment_score(x) for x in predicted_sentiment_vectors])
    
    # neutral axis:
    top_Y = np.array([x[2] for x in top_sentiments])
    pred_Y = np.array([x[2] for x in predicted_sentiment_vectors])
    
    fig_1, ax_1 = plt.subplots()#figsize=(15,10))
    plt.title("sentiment-score-plot")
    plt.xlabel("sentiment score")
    plt.ylabel("neutrality")
    plt.xlim([-1,1])
    plt.ylim([0,1])
    for i in range(len(top_X)):
        plt.text(top_X[i], top_Y[i], top_emojis[i])
    plt.plot(pred_X, pred_Y, 'bo')
    #plt.savefig(title + " -- sentiment-plot.png", bbox_inches='tight')
    
    # sentiment score axis
    top_X = np.array([x[0] for x in top_sentiments])
    pred_X = np.array([x[0] for x in predicted_sentiment_vectors])
    
    # neutral axis:
    top_Y = np.array([x[1] for x in top_sentiments])
    pred_Y = np.array([x[1] for x in predicted_sentiment_vectors])
    
    fig_2, ax_2 = plt.subplots()#figsize=(15,10))
    plt.title("positive-negative-plot")
    plt.xlabel("positive")
    plt.ylabel("negative")
    plt.xlim([0,1])
    plt.ylim([0,1])
    for i in range(len(top_X)):
        plt.text(top_X[i], top_Y[i], top_emojis[i])
    plt.plot(pred_X, pred_Y, 'bo')
    #plt.savefig(title + " -- positive-negative-plot.png", bbox_inches='tight')
    plt.show()

## testing area

In [12]:
top_20 = list("😳😋😀😌😏😔😒😎😢😅😁😉🙌🙏😘😊😩😍😭😂")
top_20_sents = twl.emoji2sent(top_20)

pred = None

def test_input(b):
    global sdm
    global pm
    global tr
    global pred
    with out_areas["playground"]:
        clear_output()
        mp("----")
        if pm is None:
            sys.stderr.write("ERROR: load or create classifier first")
            return
        X = shown_widgets["test_input"].value
        pred = pm.predict([X])
        target_list=list(shown_widgets["prediction_ground_set"].value)
        shown_widgets["prediction"].value = "<h1> " + str(twl.sent2emoji(pred,custom_target_emojis=target_list)[0]) + "</h1>"
        if shown_widgets["show_sorted_list"].value:
            mp("## " + "".join(twl.edist.sentiment_vector_to_emoji(pred, only_emoticons=True, n_results=100, custom_target_emojis=target_list)))
        


def plot_pred(b):
    global sdm
    global pm
    global tr
    global pred
    with out_areas["playground"]:
        plot_sentiment_space(pred, top_20_sents, top_20)
        
        
def plot_subset_pred(b):
    global sdm
    global pm
    global tr
    global pred
    with out_areas["playground"]:
        clear_output()
        
        if sdm is None or pm is None:
            sys.stderr.write("ERROR: sample data and/or classifier missing!\n")
            return
        
        if tr is None:
            tr = twl.trainer(sdm=sdm, pm=pm)
        
        pred, y = tr.test(emoji_subset=list(shown_widgets["validation_emojis"].value))
        print(len(pred))
        plot_sentiment_space(pred, top_20_sents, top_20)

#link
shown_widgets["test_input"].observe(test_input)
shown_widgets["show_plot"].on_click(plot_pred)
shown_widgets["show_validation_plot"].on_click(plot_subset_pred)